# 1D CNN for sEMG 17 premade gestures recognition v1.0
## 1. Import Libraries and Verify TF GPU

In [2]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras import layers, models, callbacks
from datetime import datetime
from tensorflow.python.client import device_lib
import h5py


2025-02-13 20:40:57.597787: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-13 20:40:57.694113: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-13 20:40:57.721832: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-13 20:40:57.902097: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-13 20:40:58.757517: W tensorflow/compiler/tf2

In [3]:
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
# Ensure TensorFlow uses GPU (forcing for WSL2)
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    print("Num GPUs Available: ", len(physical_devices))
    print("GPU: ", physical_devices)
    print("\nExtra Verification Info. GPU: \n\n", device_lib.list_local_devices())
    print("\n\nGPU is available. Forcing GPU usage...")
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
else:
    print("No GPU found. Exiting...")
    exit(1)



Num GPUs Available:  1
GPU:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Extra Verification Info. GPU: 

 [name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13280056311358210196
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5834276864
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17842994635355622389
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3070 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


GPU is available. Forcing GPU usage...


I0000 00:00:1739497262.614510   15548 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739497262.806812   15548 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739497262.806903   15548 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739497262.813402   15548 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739497262.813501   15548 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

## 2. Pre-Process Pt 2

In [ ]:
# Function to load HDF5-based .mat files with memory-efficient processing
# IF IN WSL 2 PLS RUN THIS CELL's CODE IN NATIVE WINDOWS, MUCH MUCH FASTER EVEN IF YOU CHANGE MEMORY LIMITS IN WSL2

def load_hdf5_mat_lazy(file_path, variable_name):
    with h5py.File(file_path, 'r') as f:
        dataset = f[variable_name]
        shape = dataset.shape  # Get shape without loading entire data
        dtype = dataset.dtype  # Get datatype
        
        # Read data in chunks to save memory and convert to numpy
        data = np.empty(shape, dtype=dtype)
        dataset.read_direct(data)  # Read directly to avoid memory duplication
        
        # MATLAB stores as column-major, so transpose to row-major (NumPy default)
        data = np.transpose(data)
        
        # Replace NaN values with 0.0 (handling in-place to save memory)
        np.nan_to_num(data, copy=False, nan=0.0)
        
    return data

# Load labels with flattening
def load_labels_lazy(file_path, variable_name):
    with h5py.File(file_path, 'r') as f:
        dataset = f[variable_name]
        data = np.empty(dataset.shape, dtype=dataset.dtype)
        dataset.read_direct(data)
    return data.flatten()

# Load training and testing datasets lazily
dataset = None
train_data = load_hdf5_mat_lazy('train_data.mat', 'train_data_reshaped')
np.save('train_data.npy', train_data)
print("Train Data Loaded. Shape:", train_data.shape)  # Expected (340646, 409, 8)
del dataset
del train_data

dataset = None
test_data = load_hdf5_mat_lazy('test_data.mat', 'test_data_reshaped')
np.save('test_data.npy', test_data)
print("Test Data Loaded. Shape:", test_data.shape)    # Expected (170323, 409, 8)
del dataset
del test_data

dataset = None
train_labels = load_labels_lazy('train_labels.mat', 'train_labels')
test_labels = load_labels_lazy('test_labels.mat', 'test_labels')

# Convert labels from 1-17 to 0-16 (for TensorFlow sparse categorical crossentropy)
train_labels -= 1
test_labels -= 1

np.save('train_labels.npy', train_labels)
np.save('test_labels.npy', test_labels)

# Clear memory by deleting unnecessary variables
del dataset
del train_labels, test_labels

print("Data processed and saved as NumPy files successfully.")


In [ ]:
# Load training data
X_train = np.load('./Training_And_Validation/train_data.npy')  # Shape: (340646, 409, 8)
y_train = np.load('./Training_And_Validation/train_labels.npy')  # Shape: (340646,)

# Load testing data
X_test = np.load('./Training_And_Validation/test_data.npy')  # Shape: (170323, 409, 8)
y_test = np.load('./Training_And_Validation/test_labels.npy')  # Shape: (170323,)


# Set sample-level activity threshold
sample_threshold = 0.05  # Adjust if necessary based on signal strength
window_rest_ratio = 0.65  # Minimum ratio of low-activity samples to label window as 'Rest'

# Analyze each window independently
rest_window_count1 = 0
for i, window in enumerate(X_train):
    # Calculate RMS across all channels for each time step
    sample_activity = np.sqrt(np.mean(window**2, axis=1))  # Shape: (409,)
    
    # Count how many samples are below the threshold
    low_activity_samples = np.sum(sample_activity < sample_threshold)
    low_activity_ratio = low_activity_samples / len(sample_activity)
    
    # Relabel window if 65% of samples are below threshold
    if low_activity_ratio >= window_rest_ratio:
        y_train[i] = 16  # Assign 'Rest' label
        rest_window_count1 += 1

print(f"Relabeled {rest_window_count1} windows as 'Rest' due to low activity.")

rest_window_count2 = 0
for i, window in enumerate(X_test):
    # Calculate RMS across all channels for each time step
    sample_activity = np.sqrt(np.mean(window**2, axis=1))  # Shape: (409,)
    
    # Count how many samples are below the threshold
    low_activity_samples = np.sum(sample_activity < sample_threshold)
    low_activity_ratio = low_activity_samples / len(sample_activity)
    
    # Relabel window if 65% of samples are below threshold
    if low_activity_ratio >= window_rest_ratio:
        y_test[i] = 16  # Assign 'Rest' label
        rest_window_count2 += 1

print(f"Relabeled {rest_window_count2} windows as 'Rest' due to low activity.")

print(X_train.shape, y_train.shape)

print(f"Training data shape: {X_train.shape}, Labels: {y_train.shape}")
print(f"Testing data shape: {X_test.shape}, Labels: {y_test.shape}")


Relabeled 67119 windows as 'Rest' due to low activity.
Relabeled 36660 windows as 'Rest' due to low activity.


In [5]:
print("Number of NaNs in X_train:", np.isnan(X_train).sum())


import numpy as np

def find_first_nan(arr):
    # Find indices where NaNs exist
    nan_indices = np.argwhere(np.isnan(arr))
    
    if len(nan_indices) == 0:
        print("No NaNs found in the dataset.")
        return None

    # Get the first occurrence
    first_nan_index = nan_indices[0]
    window_idx, row_idx, channel_idx = first_nan_index

    print(f"First NaN found at:")
    print(f" - Window index: {window_idx}")
    print(f" - Row (time step) index: {row_idx}")
    print(f" - Channel index: {channel_idx}")
    
    # Return the first NaN value for verification
    return window_idx, row_idx, channel_idx, arr[window_idx, row_idx, channel_idx]

# Check in X_train
print("Checking X_train for NaNs...")
result = find_first_nan(X_train)

if result:
    win_idx, row_idx, ch_idx, nan_value = result
    print(f"Value at [{win_idx}, {row_idx}, {ch_idx}] = {nan_value}")

# Check in X_test
print("Checking X_test for NaNs...")
result = find_first_nan(X_test)

if result:
    win_idx, row_idx, ch_idx, nan_value = result
    print(f"Value at [{win_idx}, {row_idx}, {ch_idx}] = {nan_value}")


Number of NaNs in X_train: 0
Checking X_train for NaNs...
No NaNs found in the dataset.
Checking X_test for NaNs...
No NaNs found in the dataset.


In [6]:
X_train[6,294,2] #after running NaN replacement

0.0

In [7]:
X_test[6,294,2]

-0.03252993960541661

In [8]:
y_test[40076]

4.0

In [9]:
np.unique(y_train)


array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16.])

In [10]:
np.unique(y_test)

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16.])

## 3. Load Dataset

In [5]:
# Force dataset operations to run on the CPU (GPU VRAM not enough for dataset and we're using tensorflow, we onyl want to use GPU for model wieghts and params and stuff not for data)
with tf.device('/CPU:0'):
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    del X_train, y_train  # Free memory after creating the dataset

    # Save the dataset
    train_dataset.save('train_dataset')
    del train_dataset

I0000 00:00:1739497351.688321   15548 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739497351.688434   15548 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739497351.688473   15548 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739497351.688747   15548 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-13 20:42:31.688765: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2112] Could not identify NUMA node of platform GPU id 0, defaulting to 0.  Your kernel may not have been built with NUMA support.

In [6]:
with tf.device('/CPU:0'):
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
    del X_test, y_test
    # Save the test dataset
    test_dataset.save('test_dataset')
    del test_dataset

2025-02-13 20:47:42.345093: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4458374848 exceeds 10% of free system memory.
2025-02-13 20:47:45.308260: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4458374848 exceeds 10% of free system memory.


In [56]:
BATCH_SIZE = 233
SHUFFLE_BUFFER_SIZE_TEST = 170323
SHUFFLE_BUFFER_SIZE_TRAIN = 340646

# Load the training dataset
train_dataset = tf.data.Dataset.load('train_dataset')

# Load the test dataset
test_dataset = tf.data.Dataset.load('test_dataset')

In [8]:
# seeing if data still correct structure and stuff
# Specify the index of the window and the value inside it
target_window_idx = 6
sample_idx = 294
channel_idx = 2

# Iterate through the dataset to find the specific window
for i, (window, label) in enumerate(test_dataset):
    if i == target_window_idx:
        # Convert the tensor to NumPy for easy indexing
        window_np = window.numpy()
        label_np = label.numpy()

        # Access the specific value
        specific_value = window_np[sample_idx, channel_idx]
        print(f"Value at X_test[{target_window_idx},{sample_idx},{channel_idx}]: {specific_value}")
        print(f"Label for this window: {label_np}")
        break


Value at X_test[6,294,2]: -0.053513178996377087
Label for this window: 0.0


In [57]:
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE_TRAIN).batch(BATCH_SIZE)
test_dataset = test_dataset.shuffle(SHUFFLE_BUFFER_SIZE_TEST).batch(BATCH_SIZE)

## Define 1D CNN Model Architecture

In [58]:
# Define 1D CNN model
# Define a simplified 1D CNN model
model = models.Sequential([
    # Input layer
    layers.Input(shape=(409, 8)),

    # Convolutional layer with reduced filters
    layers.Conv1D(filters=32, kernel_size=5, activation=tf.keras.layers.LeakyReLU(negative_slope=0.01), kernel_regularizer=tf.keras.regularizers.L2(0.01)),
    layers.BatchNormalization(),  # Add batch normalization
    layers.MaxPooling1D(pool_size=2),  # Pooling to reduce dimensions

    # Second convolutional layer
    layers.Conv1D(filters=64, kernel_size=5, activation=tf.keras.layers.LeakyReLU(negative_slope=0.01), kernel_regularizer=tf.keras.regularizers.L2(0.01)),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),

    # Third convolutional layer (reduced filters for simplicity)
    layers.Conv1D(filters=64, kernel_size=3, activation=tf.keras.layers.LeakyReLU(negative_slope=0.01), kernel_regularizer=tf.keras.regularizers.L2(0.01)),
    layers.BatchNormalization(),
    layers.GlobalAveragePooling1D(),  # Reduce to a fixed vector size

    # Fully connected layer with increased dropout
    layers.Dense(64, activation=tf.keras.layers.LeakyReLU(negative_slope=0.01), kernel_regularizer=tf.keras.regularizers.L2(0.01)),

    # Output layer for 17 gesture classes
    layers.Dense(17, activation='softmax')

])




In [59]:

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [60]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_15 (Conv1D)              │ (None, 405, 32)        │         1,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 405, 32)        │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_10 (MaxPooling1D) │ (None, 202, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_16 (Conv1D)              │ (None, 198, 64)        │        10,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 198, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_11 (MaxPooling1D) │ (None, 99, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_17 (Conv1D)              │ (None, 97, 64)         │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 97, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_5      │ (None, 64)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 17)             │         1,105 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,873 (116.69 KB)

 Trainable params: 29,553 (115.44 KB)

 Non-trainable params: 320 (1.25 KB)

In [61]:
# Set up TensorBoard logging
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Early stopping to prevent overfitting
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [62]:
# Train the model
model.fit(train_dataset, epochs=20, validation_data=test_dataset,
    callbacks=[tensorboard_callback],
    verbose=1,
)

Epoch 1/20


2025-02-13 22:47:44.695194: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:789: Filling up shuffle buffer (this may take a while): 160042 of 340646
2025-02-13 22:47:56.471933: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


1458/1462 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3321 - loss: 3.5465

2025-02-13 22:48:24.695520: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:791: Filling up shuffle buffer (this may take a while): 168247 of 170323
2025-02-13 22:48:24.822415: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


1462/1462 ━━━━━━━━━━━━━━━━━━━━ 58s 20ms/step - accuracy: 0.3323 - loss: 3.5445 - val_accuracy: 0.4735 - val_loss: 2.2234
Epoch 2/20


...

1462/1462 ━━━━━━━━━━━━━━━━━━━━ 48s 19ms/step - accuracy: 0.5755 - loss: 1.3483 - val_accuracy: 0.5420 - val_loss: 1.4560
Epoch 20/20


2025-02-13 23:05:08.481973: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:789: Filling up shuffle buffer (this may take a while): 164334 of 340646
2025-02-13 23:05:20.074020: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


1460/1462 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5789 - loss: 1.3379

2025-02-13 23:05:44.594727: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:791: Filling up shuffle buffer (this may take a while): 160708 of 170323
2025-02-13 23:05:45.160757: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


1462/1462 ━━━━━━━━━━━━━━━━━━━━ 51s 20ms/step - accuracy: 0.5789 - loss: 1.3380 - val_accuracy: 0.5419 - val_loss: 1.4512


In [63]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_15 (Conv1D)              │ (None, 405, 32)        │         1,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 405, 32)        │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_10 (MaxPooling1D) │ (None, 202, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_16 (Conv1D)              │ (None, 198, 64)        │        10,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 198, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_11 (MaxPooling1D) │ (None, 99, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_17 (Conv1D)              │ (None, 97, 64)         │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 97, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_5      │ (None, 64)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 17)             │         1,105 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 88,981 (347.59 KB)

 Trainable params: 29,553 (115.44 KB)

 Non-trainable params: 320 (1.25 KB)

 Optimizer params: 59,108 (230.89 KB)

# Later stuff

In [ ]:
# Save the trained model
model.save('sEMG1DCNNv2.h5')
print("Model saved as 'sEMG1DCNNv1.h5'")

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

# Launch TensorBoard command (for WSL2 use the correct path)
print("Run the following command to start TensorBoard:")
print(f"tensorboard --logdir=logs/fit --bind_all")

Model saved as 'sEMG1DCNNv1.h5'


2025-02-13 23:06:40.866071: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:791: Filling up shuffle buffer (this may take a while): 139324 of 170323
2025-02-13 23:06:43.865768: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


731/731 ━━━━━━━━━━━━━━━━━━━━ 17s 5ms/step - accuracy: 0.5423 - loss: 1.4502
Test Accuracy: 54.19%
Run the following command to start TensorBoard:
tensorboard --logdir=logs/fit --bind_all


: 